<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_PAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [26]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

In [27]:
from pandas_datareader import data as pdr
import yfinance as yf

# yfinance 라이브러리 내장 기능을 활성화한다.
# yfinance의 데이터 가져오기 기능이 pandas_datareader 를 통해 사용할 수 있다.
yf.pdr_override()

import pandas as pd
import exchange_calendars as ecals
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 11)

# ETF 가격 지정된 기간에서 가져와서 DataFrame에 담기
'''
def get_price_data(x, start_date, end_date):
  df_price = pd.DataFrame(columns=x)
  for ticker in x:
    df_price[ticker] = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)['Adj Close']
  return df_price.dropna(axis=0)
'''

# 1년 전 개장일 구하기
us_market_code = "XNYS"
us_calendar = ecals.get_calendar(us_market_code)

# 각 날짜 변수에 저장
today = datetime.today().date()
today_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=0)).date())
month_01_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=30)).date())
month_03_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=91)).date())
month_06_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=182)).date())
month_12_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=365)).date())

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
start_date = today - timedelta(days=375)

# 전략에 사용할 ETF 명 리스트에 담기
PAA = ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'IYR', 'GSG', 'GLD', 'HYG', 'LQD', 'TLT', 'IEF']

# ETF 가격 추출
# df_PAA = get_price_data(PAA, month_12_ago_workday, today).sort_index(ascending=False)
# [PAA] : PAA 순서대로 만들기
df_PAA = yf.download(PAA, start=start_date, end=today)['Adj Close'].sort_index(ascending=False)[PAA]
display(df_PAA)

[*********************100%%**********************]  13 of 13 completed


,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
Date,,,,,,,,,,,,,
2023-09-22,430.4200,357.9100,176.6700,58.6700,61.8000,38.4700,80.1900,22.5800,178.6200,74.0000,103.8400,91.4300,92.4400
2023-09-21,431.3900,357.8600,177.0700,58.8000,61.6300,38.0200,80.6800,22.5300,178.0500,73.8700,103.2100,90.7000,92.0800
2023-09-20,438.6400,364.5400,179.8600,59.6600,62.6000,38.6900,83.6300,22.5800,179.3500,74.4000,104.2600,93.0900,92.7700
2023-09-19,442.7100,369.8700,181.6800,59.5500,63.4500,38.8500,83.5400,22.8400,179.2200,74.4600,104.2200,92.8000,92.8400
2023-09-18,443.6300,370.6600,182.3900,59.5100,63.1400,39.0800,83.9200,22.8000,179.3900,74.6900,104.5700,93.4900,93.1800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-21,371.5411,281.6509,172.5204,47.1481,50.2106,36.2296,85.2295,21.3800,155.7900,69.3884,102.5184,104.7507,95.6160
2022-09-20,378.1373,286.7861,175.1967,47.7578,50.7953,36.8454,86.5399,21.4600,155.0700,69.5961,102.1337,103.0247,95.2362
2022-09-19,382.5282,289.0905,177.6171,48.8513,51.3304,37.0506,88.7043,21.5100,155.9600,70.3136,103.1627,104.0622,95.8302


In [28]:
# 12개월 이동평균 생성 및 저장
PAA_12mo_ma = df_PAA.rolling(window=252).mean().loc[df_PAA.index[251]]  # 252 개의 영업일은 대략 12개월
df_PAA_12mo_ma = pd.DataFrame([PAA_12mo_ma], columns=PAA, index=['12mo_moving_average'])

# 가격 비율 계산 및 DataFrame 생성
df_PAA_12mo_ror = pd.DataFrame(index=['12개월 수익률'])
for symbol in PAA:
    price_today = df_PAA[symbol][0]
    price_12mo_ma = PAA_12mo_ma[symbol]
    ror = (price_today / price_12mo_ma - 1) * 100
    df_PAA_12mo_ror[symbol] = [ror]

df_PAA_12mo_ma = pd.concat([df_PAA_12mo_ma, df_PAA_12mo_ror])

# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} PAA.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

str_PAA_12mo_ma = str_today + "### 12개월 이동평균 및 12개월 수익률"
display(Markdown(str_PAA_12mo_ma))
display(df_PAA_12mo_ma)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'w') as f:
  f.write(f'{str_PAA_12mo_ma}\n')
  df_PAA_12mo_ma.to_csv(f, index=True)

# 0 이하 원소 개수 세기
count_negative_etf = (df_PAA_12mo_ror < 0).sum().sum().item()
str_count_negative_etf = "### 12개월 수익률 0 미만 ETF 개수: " + str(count_negative_etf)

display(Markdown(str_count_negative_etf))

# 안전자산 비율 계산
display(Markdown('### 안전자산 비율: ' + str(round((count_negative_etf) * (100/6), 1)) + '%'))

### 2023-09-25 ### 12개월 이동평균 및 12개월 수익률

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
12mo_moving_average,406.7717,316.6860,180.8297,57.3854,57.2979,38.4867,84.5520,20.7100,174.2733,72.3973,104.6186,100.0571,95.3953
12개월 수익률,5.8137,13.0173,-2.3003,2.2386,7.8573,-0.0434,-5.1589,9.0295,2.4942,2.2137,-0.7443,-8.6221,-3.0980


### 12개월 수익률 0 미만 ETF 개수: 6

### 안전자산 비율: 100.0%

In [29]:
# 모멘텀스코어 계산할 오늘-1-3-6-12개월 날짜 저장
ago_name = ['today', 'month_01_ago', 'month_03_ago', 'month_06_ago', 'month_12_ago']
ago_date = [today_workday.date(), \
            month_01_ago_workday.date(), \
            month_03_ago_workday.date(), \
            month_06_ago_workday.date(), \
            month_12_ago_workday.date()]

# '개월 전/날짜' 형식으로 텍스트 저장
ago = [f'{a}/{d}' for a, d in zip(ago_name, ago_date)]

# ETF의 날짜별 데이터 담기
etf_data = {}

for i in range(len(PAA)):
  etf_data[PAA[i]] = [df_PAA.loc[today_workday, PAA[i]], \
                          df_PAA.loc[month_01_ago_workday, PAA[i]], \
                          df_PAA.loc[month_03_ago_workday, PAA[i]], \
                          df_PAA.loc[month_06_ago_workday, PAA[i]], \
                          df_PAA.loc[month_12_ago_workday, PAA[i]]]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
df_etf_price = pd.DataFrame(etf_data, columns=PAA, index=ago)
str_etf_price = str_today + "### ETF 가격 현재-1-3-6-12 개월"

display(Markdown(str_etf_price))
display(df_etf_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_price}\n')
  df_etf_price.to_csv(f, index=True)

### 2023-09-25 ### ETF 가격 현재-1-3-6-12 개월

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
today/2023-09-22,430.4200,357.9100,176.6700,58.6700,61.8000,38.4700,80.1900,22.5800,178.6200,74.0000,103.8400,91.4300,92.4400
month_01_ago/2023-08-25,438.4235,363.4938,184.0000,59.7498,59.9700,38.9100,83.7900,21.5000,177.6200,74.2621,104.5873,94.9352,93.4508
month_03_ago/2023-06-23,431.6873,362.0160,180.5700,59.9390,60.9100,39.1700,82.2400,19.5300,178.2000,73.0994,106.6062,102.4604,96.3753
month_06_ago/2023-03-24,392.8995,310.0154,171.3156,57.0391,56.7394,38.3694,80.3109,19.2400,183.6500,71.5351,107.3074,105.1373,98.4778
month_12_ago/2022-09-23,362.2474,273.6551,164.6196,45.0094,49.5863,35.1643,83.2786,20.5300,153.0100,68.0950,100.7875,102.4914,94.2915


In [30]:
etf_momentum_score = []
etf_ror_data = []

## DataFrame 만들기: 수익률
for etf_name in PAA:
  etf_today_price = df_etf_price.loc[ago[0], etf_name]
  etf_momentum_score.append((12 * (etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100) + \
                            (4 * (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100) + \
                            (2 * (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100) + \
                            (1 * (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100))

  etf_ror_data.append([(etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100])

df_etf_ror = pd.DataFrame(etf_ror_data, columns=ago[1:], index=PAA).transpose()

str_etf_ror = str_today + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)
  # df_etf_ror.to_csv(filename, mode='a', header=True, index=True)

### 2023-09-25 ### ETF 1-3-6-12 개월 수익률

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
month_01_ago/2023-08-25,-1.8255,-1.5361,-3.9837,-1.8072,3.0515,-1.1308,-4.2965,5.0233,0.5630,-0.3530,-0.7145,-3.6923,-1.0817
month_03_ago/2023-06-23,-0.2936,-1.1342,-2.1598,-2.1171,1.4612,-1.7871,-2.4927,15.6170,0.2357,1.2320,-2.5948,-10.7655,-4.0833
month_06_ago/2023-03-24,9.5496,15.4491,3.1255,2.8593,8.9190,0.2622,-0.1505,17.3597,-2.7389,3.4457,-3.2313,-13.0375,-6.1311
month_12_ago/2022-09-23,18.8193,30.7887,7.3202,30.3506,24.6311,9.4006,-3.7088,9.9854,16.7375,8.6717,3.0287,-10.7925,-1.9636


In [31]:
## DataFrame 만들기: 수익률 가중치
df_etf_ror_weight = df_etf_ror
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 12 if row.name == ago[1] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 4 if row.name == ago[2] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 2 if row.name == ago[3] else row, axis=1)

str_etf_ror_weight = str_today + "### ETF 1-3-6-12 개월 가중치 적용"
display(Markdown(str_etf_ror_weight))
display(df_etf_ror_weight)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror_weight}\n')
  df_etf_ror_weight.to_csv(f, index=True)

### 2023-09-25 ### ETF 1-3-6-12 개월 가중치 적용

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
month_01_ago/2023-08-25,-21.9062,-18.4338,-47.8044,-21.6861,36.6183,-13.5697,-51.5574,60.2791,6.7560,-4.2356,-8.5738,-44.3070,-12.9801
month_03_ago/2023-06-23,-1.1743,-4.5368,-8.6393,-8.4683,5.8447,-7.1483,-9.9708,62.4680,0.9428,4.9281,-10.3791,-43.0620,-16.3332
month_06_ago/2023-03-24,19.0993,30.8982,6.2510,5.7186,17.8380,0.5244,-0.3010,34.7193,-5.4778,6.8913,-6.4626,-26.0751,-12.2622
month_12_ago/2022-09-23,18.8193,30.7887,7.3202,30.3506,24.6311,9.4006,-3.7088,9.9854,16.7375,8.6717,3.0287,-10.7925,-1.9636


In [32]:
## DataFrame 만들기: 모멘텀
df_etf_momentum_score = pd.DataFrame(etf_momentum_score, columns=['momentum_score'], index=PAA).transpose()

str_etf_momentum_score = str_today + "### ETF 별 12개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.to_csv(f, index=True)

### 2023-09-25 ### ETF 별 12개월 Momentum Score

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
momentum_score,14.8381,38.7164,-42.8726,5.9148,84.9321,-10.7931,-65.5380,167.4518,18.9584,16.2555,-22.3868,-124.2367,-43.5390


In [33]:
# 12개월 수익률 (-)인 ETF 개수 및 16.7%씩 투자할 자산 개수와 종목
count_invest_etf = (lambda count_negative_etf: 6 - count_negative_etf if count_negative_etf < 7 else 0)(count_negative_etf)
str_count_invest_etf = "## 16.7% 씩 투자할 자산 개수: " + str(count_invest_etf)

copy_etf_momentum_score = etf_momentum_score.copy()
recommended_etfs = []
for i in range(count_invest_etf):
  recommended_etfs.append(PAA[copy_etf_momentum_score.index(max(copy_etf_momentum_score))])
  copy_etf_momentum_score[copy_etf_momentum_score.index(max(copy_etf_momentum_score))] = -99999  # 선택된 ETF 점수를 낮은 값으로 변경

str_etf_PAA = "## 및 추천 종목: " + ' '.join(recommended_etfs)
str_etf_ratio = "## 투자 비율 - 현금대체재 | 보호막 자산: " + str(round(count_negative_etf * 1/6 * 100, 1)) + "% | " + str(round(count_invest_etf * 1/6 * 100, 1)) + "%"

display(Markdown(str_today[1:]))
display(Markdown(str_count_negative_etf[1:]))
display(Markdown(str_count_invest_etf))
display(Markdown(str_etf_PAA))
display(Markdown(str_etf_ratio))

# csv에 자산 투자 조언 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:]}\n')
  f.write(f'{str_count_negative_etf[1:]}\n')
  f.write(f'{str_count_invest_etf}\n')
  f.write(f'{str_etf_PAA}\n')
  f.write(f'{str_etf_ratio}')

## 2023-09-25 

## 12개월 수익률 0 미만 ETF 개수: 6

## 16.7% 씩 투자할 자산 개수: 0

## 및 추천 종목: 

## 투자 비율 - 현금대체재 | 보호막 자산: 100.0% | 0.0%